In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta
from SML import training_pipeline

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549016
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


../src/arrow/status.cc:137: DoAction result was not fully consumed: Cancelled: Flight cancelled call, with message: CANCELLED. Detail: Cancelled


Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549016/jobs/named/tesla_stocks_fv_1_create_fv_td_07052024090051/executions
2024-05-07 11:02:21,906 WARNING: VersionWarning: Incremented version to `1`.

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5555
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/528 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/549016/models/stock_pred_model/7


In [2]:
project = hopsworks.login()
fs= project.get_feature_store()
mr = project.get_model_registry() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549016
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
start_date = datetime.now() - timedelta(hours=24)
print(start_date.strftime("%Y-%m-%d"))

2024-05-06


In [12]:
end_date = datetime.now().strftime("%Y-%m-%d")
print(end_date)

2024-05-07


In [13]:
feature_view = fs.get_feature_view('tesla_stocks_fv', 1)
feature_view.init_batch_scoring(training_dataset_version=1)

In [14]:
print(feature_view.get_batch_query())

WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`ticker` `ticker`, `fg1`.`ticker` `join_pk_ticker`, `fg1`.`date` `join_evt_date`, `fg0`.`sentiment` `sentiment`, RANK() OVER (PARTITION BY `fg1`.`ticker`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `mtzeve_featurestore`.`tesla_stock_1` `fg1`
INNER JOIN `mtzeve_featurestore`.`news_sentiment_updated_1` `fg0` ON `fg1`.`ticker` = `fg0`.`ticker` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`date` `date`, `right_fg0`.`ticker` `ticker`, `right_fg0`.`sentiment` `sentiment`
FROM right_fg0)


In [16]:
# we had problems fetching the data from fv with get_batch_data function, tried everything and it just doesnt work 
tsla_df_b = feature_view.get_batch_data(start_time = start_date, end_time = end_date)
tsla_df_b

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.11s) 


,date,ticker,sentiment


In [11]:
import joblib
the_model = mr.get_model("stock_pred_model", version=7)
model_dir = the_model.download()

model = joblib.load(model_dir + 'stock_model')

FileNotFoundError: [Errno 2] No such file or directory: '/var/folders/ty/fy7wpfqs4c39hnsfl21_rzyc0000gn/T/d6edbe1d-de39-488f-b12c-c0cbfd5ded37/stock_pred_model/7stock_model'

In [ ]:
predictions = model.predict(tsla_df_b)

In [ ]:
predictions 